## All in One

In [2]:
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np
from pathlib import Path
import pandas as pd

#filename = "aisdk-2023-11-08_1_knn"
filename = "aisdk-2023-11-09_1_knn"

data =  pd.read_csv(f"../workdir/AIS-KNN-Files/{filename}.csv", lineterminator='$').values.reshape((-1,100,6))

multiindex = pd.MultiIndex.from_product([range(s) for s in data.shape], names=["track", "timeindex", "column"])
data = pd.Series(data.flatten(), index=multiindex).unstack(level="column")
data.columns = ["mmsi", "time", "x", "y", "deg", "dist"]
data = data.drop(columns=["deg", "dist"])
data["time"] = pd.to_datetime(data["time"], unit="s")
train_ships = pd.Series(data.mmsi.unique()).sample(frac=0.7, random_state=42)
train = data[data.mmsi.isin(train_ships)].drop(columns="mmsi")
test = data[~data.mmsi.isin(train_ships)].drop(columns="mmsi")

data = []
train_ships = []

def generate_row(df):
    df = df.copy()
    df["src"] = "output"
    df["time"] = (df["time"].max()-df["time"]).dt.total_seconds()
    df["x"] = df["x"] - df["x"].iloc[0]
    df["y"] = df["y"] - df["y"].iloc[0]
    df.iloc[:-1,-1] = "input_" + pd.Series(np.arange(df.shape[0]-1)).astype(str)
    df=df.melt(id_vars="src")
    df["colname"] = df.src + "_" + df.variable
    df=df.drop(columns=["src", "variable"]).set_index("colname")
    df=df.transpose().reset_index(drop=True)
    df.columns.name = None
    df=df.sort_index(axis=1).drop(["output_time"], axis=1)
    df=df.loc[:, ~df.columns.to_series().str.endswith('index')]
    return df

def generate_model_frame(df, known):
    window_size = known + 1
    skipped_windows = 4

    windows = df.reset_index(drop=False).groupby('track').rolling(window_size)
    windows = (win for i, win in enumerate(windows) if len(win) == window_size and i % (skipped_windows + 1) == 0)
    windows = list(windows)
    res = Parallel(n_jobs=-1)(delayed(generate_row)(win) for win in tqdm(windows))
    return pd.concat(res, ignore_index=True)

for known in range(12, 2, -1):
    path = Path(f"../workdir/AIS-ModelFrames/{filename}_train_{known}_0.csv")
    if not path.exists():     
        train_model = generate_model_frame(train, known)
        test_model = generate_model_frame(test, known)

        if known == 12:   
            var = list(train_model.shape)
            var = var[0]
            trainlimit = var
            print(f"Trainlimit: {trainlimit}")

            var = list(test_model.shape)
            var = var[0]
            testlimit = var
            print(f"Testlimit: {testlimit}")

        print(train_model.shape)
        train_model = train_model.sample(n = trainlimit)
        print(train_model.shape)

        print(test_model.shape)
        test_model = test_model.sample(n = testlimit)
        print(test_model.shape)
        
        train_model.to_csv(path, index=False)
        test_model.to_csv(f"../workdir/AIS-ModelFrames/{filename}_test_{known}_0.csv", index=False)
        print(f"Done with {known}")

100%|██████████| 1264800/1264800 [13:49<00:00, 1524.34it/s]


Trainlimit: 2838847
Testlimit: 1264800
(2838847, 38)
(2838847, 38)
(1264800, 38)
(1264800, 38)
Done with 12


100%|██████████| 1264800/1264800 [13:41<00:00, 1540.29it/s]


(2838847, 35)
(2838847, 35)
(1264800, 35)
(1264800, 35)
Done with 11


100%|██████████| 1339200/1339200 [14:28<00:00, 1542.09it/s]


...

(3172829, 17)
(2838847, 17)
(1413600, 17)
(1264800, 17)
Done with 5


100%|██████████| 1413600/1413600 [14:42<00:00, 1601.86it/s]


(3172829, 14)
(2838847, 14)
(1413600, 14)
(1264800, 14)
Done with 4


100%|██████████| 1413600/1413600 [14:45<00:00, 1596.53it/s]


(3172829, 11)
(2838847, 11)
(1413600, 11)
(1264800, 11)
Done with 3
